In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import glob
import os

In [153]:
def read_stimuli_json(subject):
    with open(f'data/{subject}/stimuli.js', 'r') as f:
        stimuli_js = f.read()
    stimuli_json_str = stimuli_js[20:]
    with open(f'data/{subject}/stimuli.json', 'w') as f:
        f.write(stimuli_json_str)
    stimuli_json = pd.read_json(f'data/{subject}/stimuli.json')
    stimuli_json.loc[:, 'triad_index'] = np.arange(1, len(stimuli_json)+1, dtype=int)
    return stimuli_json 

In [168]:
def add_triad_index_from_targets(stimuli_df, stimuli_json):
    stimuli_df_no_blanks = stimuli_df[stimuli_df.target1_rank.notnull()]
    stimuli_df_only_blanks = stimuli_df[stimuli_df.target1==0]
    stimuli_json_no_blanks = stimuli_json[stimuli_json.target1_rank.notnull()]
    triad_index_dict = {(int(target1), int(target2), int(distractor)):int(triad_index) for i, 
                    (triad_index, target1, target2, distractor) in 
                    stimuli_json_no_blanks[['triad_index', 'target1_rank', 'target2_rank', 'distractor_rank']].iterrows()}
    triad_index_dict[(0, 0, 0)] = len(stimuli_json_no_blanks) + 1
    # blank_id = 0
    for i, row in stimuli_df.iterrows():
        indices = row[['target1_rank', 'target2_rank', 'distractor_rank']]
        if all(indices.notna()):
            indices_tuple = tuple(indices.values)
            stimuli_df.loc[i, 'triad_index'] = triad_index_dict[indices_tuple]
        else:
            stimuli_df.loc[i, 'triad_index'] = triad_index_dict[(0, 0, 0)] # + blank_id
            # blank_id += 1
    stimuli_df.triad_index = stimuli_df.triad_index.astype(int)
    return stimuli_df

In [31]:
def get_durations_multiple_files(file_path):
    run_df = pd.read_csv(file_path)
    responses_df = run_df[run_df.part=='trinary']
    responses_df = responses_df[['triad_index', 'response']]
    # nas_responses = responses_df[responses_df.response.isna()]
    responses_df = responses_df.dropna()

    scan_start = run_df.index[run_df.part=='scan_wait'][0]
    run_df = run_df.iloc[scan_start:, :] # filter only scan part
    run_durations = np.diff(run_df.time_elapsed)/1000
    run_df.loc[:, 'duration'] = np.append(0, run_durations)
    elapsed_time = run_df.duration.cumsum()[:-1]
    run_df.loc[:, 'run_elapsed_time'] = np.append(0, elapsed_time)
    ev_times = run_df.loc[run_df.ID.notnull(), ['run_elapsed_time', 'duration', 'triad_index', 
                                                'response', 'rt']].reset_index(drop=True)
    ev_times.loc[:, 'ID'] = ev_times.ID.astype(int)
    

    return ev_times

In [3]:
def read_run_df(file_path):
    run_df = pd.read_csv(file_path)
    scan_start_time = run_df.loc[run_df.part=='scan_wait', 'time_elapsed']
    responses_df = run_df[run_df.part=='evaluation']
    responses_df = responses_df[['ID', 'time_elapsed', 'response', 'rt', 'prob', 'amount']]
    responses_df.loc[:, 'time_elapsed'] = responses_df.time_elapsed - scan_start_time
    responses_df.response = responses_df.response.astype(float)
    responses_df.rt = responses_df.rt/1000
    responses_df.loc[:, 'EV'] = responses_df.prob * responses_df.amount / 100
    return responses_df

In [4]:
def plot_subject_stats(all_runs, subject_num):
    fig, axs = plt.subplots(1, 3, figsize=(15, 5), dpi=150)
    # choices
    axs[0].hist(all_runs.response.dropna(), color='lightcoral')
    axs[0].set_title('Choice distribution', fontsize=18)
    axs[0].set_xlabel('Choice (NIS)', fontsize=16)
    axs[0].set_xlim(-5, 85)
    axs[0].tick_params(axis='x', labelsize=13)
    axs[0].tick_params(axis='y', labelsize=13)
    # RT and misses
    n_blank_trials = len(all_runs[all_runs.amount==0])
    missed_trials = all_runs.rt.isna().sum() - n_blank_trials
    axs[1].hist(all_runs.rt, color='darkseagreen')
    axs[1].set_title(f'RT distribution (missed: {missed_trials})', fontsize=18)
    axs[1].set_xlabel('RT (s)', fontsize=16)
    axs[1].set_xlim(0, 5.5)
    axs[1].tick_params(axis='x', labelsize=13)
    axs[1].tick_params(axis='y', labelsize=13)
    # EV sensitivity
    ev_sv_corr = all_runs[['EV', 'response']].corr().iloc[0, 1] # ignore nans
    axs[2].scatter(all_runs.EV, all_runs.response, color='steelblue', alpha=0.5)
    axs[2].set_title(f'EV sensitivity (r={ev_sv_corr:.2f})', fontsize=18)
    axs[2].set_xlabel('EV', fontsize=16)
    axs[2].set_ylabel('Choice', fontsize=16)
    axs[2].set_ylim(-5, 85)
    axs[2].tick_params(axis='x', labelsize=13)
    axs[2].tick_params(axis='y', labelsize=13)
    plt.suptitle(f'Subject {subject_num}', fontsize=18)

In [5]:
def get_durations(mri_df, run_i):
    # start and end indices of the run
    n_runs = len(np.where(mri_df.part=='scan_wait')[0])
    run_indices = np.where(mri_df.part=='scan_wait')[0][run_i-1:run_i+1]

    if run_i==n_runs:
        last_delay_ind = np.where(mri_df.part=='fixation')[0][-1]
        run_indices = np.append(run_indices, last_delay_ind)

    # only indices of run i
    run_df = mri_df.iloc[run_indices[0]:run_indices[1], :].copy()
    run_durations = np.diff(run_df.time_elapsed)/1000
    run_df.loc[:, 'duration'] = np.append(0, run_durations)
    elapsed_time = run_df.duration.cumsum()[:-1]
    run_df.loc[:, 'run_elapsed_time'] = np.append(0, elapsed_time)
    ev_times = run_df.loc[run_df.ID.notnull(), ['run_elapsed_time', 'duration', 'ID', 'part']].reset_index(drop=True)
    ev_times.loc[:, 'ID'] = ev_times.ID.astype(int)

    # sum duration of viewing and evaluation task
    merged_view_eval_times = pd.DataFrame(columns=ev_times.columns[:-1])
    for i in range(0, len(ev_times), 2):
        eval_duration = ev_times.loc[i+1, 'duration']
        view_duration = ev_times.loc[i, 'duration']
        merged_view_eval_times.loc[i, :] = ev_times.loc[i, :]
        merged_view_eval_times.loc[i, 'duration'] = view_duration + eval_duration
    merged_view_eval_times = merged_view_eval_times.reset_index(drop=True)
    merged_view_eval_times.loc[:, 'ev'] = np.arange(1, len(merged_view_eval_times)+1)

    return ev_times, merged_view_eval_times

In [128]:
def write_evs_to_txt(merged_view_eval_times, run_i, fsf_dir):
    """
    Write EVs to txt files for each run
    """
    i = 1
    triad_indices = merged_view_eval_times.triad_index.unique()
    for triad_index in triad_indices:
        triad_df = merged_view_eval_times[merged_view_eval_times.triad_index==triad_index]
        if triad_index == max(triad_indices):
            # blank trials
            start_str = [str(np.round(run_elapsed_time, 3)) for run_elapsed_time in triad_df.time]
            duration_str = [str(np.round(duration, 3)) for duration in triad_df.duration]
            ev_string = [' '.join([start_str, duration_str, '1']) for start_str, duration_str in zip(start_str, duration_str)]
            ev_string = '\n'.join(ev_string)
        else:
            elapsed_str = str(merged_view_eval_times.loc[i, 'time'])
            duration_str = str(merged_view_eval_times.loc[i, 'duration'])
            ev_string = ' '.join([elapsed_str, duration_str, '1'])        
        with open(fsf_dir + f'/run{run_i}/evs/run{run_i}_ev{i}.txt', 'w') as f:
            f.write(ev_string) 
        i += 1

In [129]:
def write_evs_to_txt_unify_blanks(merged_view_eval_times, run_i, fsf_dir):
    """
    Write EVs to txt files for each run
    """
    i = 1
    for lottery_id in merged_view_eval_times.ID.unique():
        lottery_df = merged_view_eval_times[merged_view_eval_times.ID==lottery_id] 
        if lottery_id == 0:
            start_str = [str(np.round(run_elapsed_time, 3)) for run_elapsed_time in lottery_df.run_elapsed_time]
            duration_str = [str(np.round(duration, 3)) for duration in lottery_df.duration]
            ev_string = [' '.join([start_str, duration_str, '1']) for start_str, duration_str in zip(start_str, duration_str)]
            ev_string = '\n'.join(ev_string)
        else:
            start_str = str(np.round(lottery_df.run_elapsed_time.values[0], 3))
            duration_str = str(np.round(lottery_df.duration.values[0], 3))
            ev_string = ' '.join([start_str, duration_str, '1'])
        with open(fsf_dir + f'/run{run_i}/evs_unified_blanks/run{run_i}_ev{i}.txt', 'w') as f:
            f.write(ev_string)
        i += 1

In [8]:
def write_evs_to_txt_unify_view(unified_trials, run_i, fsf_dir):
    """
    Write EVs to txt files for each run
    """
    i = 1
    for lottery_id in unified_trials.ID.unique():
        lottery_df = unified_trials[unified_trials.ID==lottery_id] 
        if lottery_id == 0:
            start_str = [str(np.round(view_time, 3)) for view_time in lottery_df.view_time]
            duration_str = [str(np.round(rt + 1, 3)) for rt in lottery_df.rt]
            ev_string = [' '.join([start_str, duration_str, '1']) for start_str, duration_str in zip(start_str, duration_str)]
            ev_string = '\n'.join(ev_string)
        else:
            start_str = str(np.round(lottery_df.view_time.values[0], 3))
            # duration_str = str(np.round(lottery_df.view_duration.values[0], 3))
            duration_str = "1"
            ev_string = ' '.join([start_str, duration_str, '1'])
        with open(fsf_dir + f'/run{run_i}/view/evs_view/run{run_i}_ev{i}.txt', 'w') as f:
            f.write(ev_string)
        i += 1

### Which EVs to average?

In [182]:
SUBJECT_NUM = '104'
results_dir = f'data/{SUBJECT_NUM}/'
results_path = glob.glob(results_dir + '*.csv')
ignore_choice_results = ['neuro_DN' not in f for f in results_path]
results_path = np.array(results_path)[ignore_choice_results]
all_runs = pd.DataFrame()
WRITE_EVS = True
df_list = []
stimuli_json = read_stimuli_json(SUBJECT_NUM)
if WRITE_EVS:
    fsf_dir = f'/Volumes/homes/Asaf/qunex/neuro_DN/task_analysis/{SUBJECT_NUM}'
    if not os.path.exists(fsf_dir):
            os.mkdir(fsf_dir)
ev_ids = []
for f in results_path:
    # extract stimuli ids
    regex_files = re.findall(r"/(\d)_", f)
    if len(regex_files) > 0:
        run_i = re.findall(r"/(\d)_", f)[0]
    else:
        run_i = 1
    print(f'Processing run {run_i}...')
    this_run_df = pd.read_csv(f)
    scan_start = this_run_df.index[this_run_df.part=='scan_wait'][0]
    this_run_df = this_run_df.iloc[scan_start:, :] # filter only scan part
    this_run_df.loc[:, 'duration'] = this_run_df.time_elapsed.diff() / 1000
    this_run_df.loc[:, 'rt'] = this_run_df.rt / 1000
    times = this_run_df.duration.cumsum()[:-1].fillna(0)
    times = np.append(np.nan, times)
    this_run_df.loc[:, 'time'] = times
    # remove all fixation/non-trials
    this_run_df = this_run_df[this_run_df.target1.notnull()]
    # add triad index, which are missing from pilot subjects...
    this_run_df = add_triad_index_from_targets(this_run_df, stimuli_json)
    this_run_df.loc[:, 'run'] = int(run_i) 
    # reset index for trial order and number
    this_run_df.reset_index(drop=True, inplace=True)
    if WRITE_EVS:
        if not os.path.isdir(fsf_dir + f'/run{run_i}/'):
            os.mkdir(fsf_dir + f'/run{run_i}/')    
        if not os.path.isdir(fsf_dir + f'/run{run_i}/evs/'):
            os.mkdir(fsf_dir + f'/run{run_i}/evs/')
        write_evs_to_txt(this_run_df, run_i, fsf_dir)
    trial_order = np.arange(1, len(this_run_df)+1)
    temp_ev_id = pd.DataFrame({f'stimulus_id':this_run_df.triad_index, f'ev_num_run{run_i}':trial_order})
    temp_ev_id = temp_ev_id.drop_duplicates('stimulus_id').reset_index(drop=True)
    temp_ev_id.loc[:, f'ev_num_run{run_i}'] = np.arange(1, len(temp_ev_id)+1)
    ev_ids.append(temp_ev_id)
    df_list.append(this_run_df)
all_runs = pd.concat(df_list)

Processing run 4...
Processing run 2...
Processing run 1...
Processing run 3...


In [183]:
merged_evs = pd.DataFrame({f'stimulus_id':[]})
for ev_id_df in ev_ids:
    merged_evs = pd.merge(merged_evs, ev_id_df, on='stimulus_id', how='outer')
merged_evs_sorted = merged_evs.sort_values('stimulus_id').set_index('stimulus_id')
merged_evs_sorted

,ev_num_run4,ev_num_run2,ev_num_run1,ev_num_run3
stimulus_id,,,,
1,12,33,18,25
2,36,11,24,21
3,13,28,8,17
4,11,4,13,32
5,9,19,6,29
6,20,34,31,13
7,22,10,28,10
8,33,22,23,9
9,32,24,12,24


In [184]:
merged_evs_sorted.to_csv(f'ev_data/{SUBJECT_NUM}_ev_ids.csv', float_format='%.0f')

### Edit fsf file

In [185]:
WRITE_FSF = True

In [186]:
if WRITE_FSF:
    fsf_path = 'ev_data/run1_hp200_s4_level1.fsf'
    fsf = open(fsf_path, 'r').read()
    for run_i in range(1, 6):
        fsf_dir = f'/Volumes/homes/Asaf/qunex/neuro_DN/task_analysis/{SUBJECT_NUM}'
        new_fsf_path = f'{fsf_dir}/run{run_i}/run{run_i}_hp200_s4_level1.fsf'
        new_fsf = fsf
        if f'ev_num_run{run_i}' not in merged_evs_sorted.columns:
            print(f'run {run_i} not available, skipping')
            continue
        else:
            run_evs = merged_evs_sorted.loc[:, f'ev_num_run{run_i}'].astype(int)
        for stimulus_id in run_evs.index:
            stimulus_ev = run_evs.loc[stimulus_id]
            line_pattern = re.compile(fr'set fmri\(custom{stimulus_id}\) ".*txt"')
            line_to_replace = re.findall(line_pattern, new_fsf)[0]
            new_line = fr'set fmri(custom{stimulus_id}) "../evs/run{run_i}_ev{stimulus_ev}.txt"'
            new_fsf = new_fsf.replace(line_to_replace, new_line)
            new_fsf = new_fsf.replace("MNINonLinear/Results/1/1", f"MNINonLinear/Results/{run_i}/{run_i}")
            new_fsf = new_fsf.replace("105", SUBJECT_NUM)
        if not os.path.exists(fsf_dir):
            os.mkdir(fsf_dir)
        open(new_fsf_path, f'w').write(new_fsf)

run 5 not available, skipping


In [115]:
if EV_TYPE == 'org_evaluation':
    fsf_path = '/Volumes/homes/Asaf/qunex/decoy/task_analysis/001/design_run1_hp200_s4_level1.fsf'
    fsf = open(fsf_path, 'r').read()
    for run_i in range(1, 6):
        fsf_dir = f'/Volumes/homes/Asaf/qunex/decoy/task_analysis/{SUBJECT_NUM}'
        new_fsf_path = f'{fsf_dir}/design_run{run_i}_hp200_s4_level1.fsf'
        new_fsf = fsf
        run_evs = merged_evs_sorted.loc[:, f'ev_num_run{run_i}']
        for stimulus_id in run_evs.index:
            stimulus_ev = run_evs.loc[stimulus_id]
            line_pattern = re.compile(f'set fmri\(custom{stimulus_id}\) ".*txt"')
            line_to_replace = re.findall(line_pattern, new_fsf)[0]
            new_line = f'set fmri(custom{stimulus_id}) "../evs/run{run_i}_ev{stimulus_ev}.txt"'
            new_fsf = new_fsf.replace(line_to_replace, new_line)
            new_fsf = new_fsf.replace("MNINonLinear/Results/1/1", f"MNINonLinear/Results/{run_i}/{run_i}")
            new_fsf = new_fsf.replace("001", SUBJECT_NUM)
        if not os.path.exists(fsf_dir):
            os.mkdir(fsf_dir)
        open(new_fsf_path, 'w').write(new_fsf)
    